In [1]:
from dotenv import load_dotenv
load_dotenv('config.env')

True

In [2]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
llm.predict("How old are you?")

'\n\nI am 20 years old.'

<H1> Chat Models </H1>

<ul>
    <li>Chat model은 언어 모델의 variation이다.</li>
    <li>Chat model은 내부적으로 언어 모델을 사용하지만, 인터페이스가 조금 다르다.</li>
    <ul>
        <li>"text in, text out" API 대신, "chat messages"가 입력과 출력인 인터페이스를 사용한다.</li>
    </ul>
</ul>

<ul>
    <li>한개 이상의 message를 chat model에게 전달해서 채팅을 진행할 수 있다.</li>
    <li>응답은 마찬가지로 message 이다.</li>
    <li>LangChain에서 현재 지원 되는 message의 type은 AIMessage, HumanMessage, SystemMessage, ChatMessage가 있다.</li>
    <li>ChatMessage는 임의의 역할 파라미터(arbitrary role parameter)를 사용한다.</li>
    <li>대부분의 경우, HumanMeesage, AIMessage, SystemMessage만 다룰 것이다.</li>
</ul>

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)
chat.predict_messages([HumanMessage(content="Translate this sentence from English to Korean. I love programming.")])

AIMessage(content='저는 프로그래밍을 좋아합니다.', additional_kwargs={}, example=False)

<H1> Prompt templates </H1>

<ul>
    <h2>LLM</h2>
    <li>대부분의 LLM application들은 user input을 바로 LLM에 전달하지 않는다.</li>
    <li>주로 prompt template이라는 더 큰 텍스트에 user input을 넣어서 추가적인 context를 전달한다.</li>
</ul>

<ul>
    <h2>Chat models</h2>
    <li>Chat model에서는 MessagePromptTemplate을 이용해서 할 수 있다.</li>
    <li>그리고 하나 이상의 MessagePromptTemplate을 이용해서 ChatPromptTemplate을 만들 수 있다.</li>
    <li>ChatPromptTemplate의 format_messages 메소드를 이용해서 formatted messages를 만들 수 있다.</li>
    <ul>
        <li>이것은 list of messages를 만드는 것으로, 단순한 string으로 만들어진 일반적인 prompt template보다 더 복잡하기 때문.</li>
    </ul>
</ul>

In [4]:
# LLM
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")



# Chat models
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# 1. template를 만들고, system message prompt로 변환한다.
template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
# 2. human_template를 만들고, human message prompt로 변환한다.
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 3. system message prompt와 human message prompt를 이용해서 chat prompt를 만든다.
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 4. chat prompt에 입력을 넣어서 결과를 받는다.
chat_prompt.format_messages(input_language="English", output_language="Korean", text="I love programming.")


# 얘는 prompt일 뿐, 아직 chat model에 넣어서 결과값을 출력한 것이 아님. Chain을 이용해보자!

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

<H1> Chain </H1>

<ul>
    <li>Chain을 이용해서 model과 prompt template을 합쳐보자.</li>
    <li>Chain은 model, prompt, 다른 chain과 같은 여러 primitive를 연결할 수 있도록 해준다. </li>
</ul>

In [5]:
# LLM
from langchain.chains import LLMChain

llm.predict("What would be a good company name for a company that makes colorful socks?")
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

chain = LLMChain(llm=llm, prompt=prompt)
chain.run("colorful socks")



# Chat models
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# 1. Chat model 만들기
chat = ChatOpenAI(temperature=0)

# 2. Prompt template 만들기
template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. Chain으로 2개를 엮어서 결과값을 출력하기
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="Korean", text="I love programming.")

'저는 프로그래밍을 좋아합니다.'

<H1>Agents</H1>

<ul>
    <li>위에서 만든 chain은 미리 결정된 형태로만 사용할 수 있다.</li>
    <li>복잡한 워크플로우를 해결하려면, 입력에 따라서 동적으로 action을 선택할 수 있어야 한다.</li>
    <li>Agent가 이걸 가능하게 한다: 언어 모델을 사용해서 어떤 조치를 취하고, 어떤 순서로 수행할지 결정한다.</li>
    <li>Agent는 tool에 대한 접근 권한을 부여받고, 반복적으로 tool을 선택하고 실행하며 최종 답변이 나올 때까지 결과를 관찰한다.</li>
</ul>

<ul>
    <li>Agent를 위해서는 3개를 선택해야 한다.</li>
    <ol>
        <li>LLM/Chat Model: Agent를 지원하는 언어 모델</li>
        <li>Tool(s): 특정한 임무를 수행하는 function. ex) Google Search, Database lookup, Python REPL, 다른 chain들. 미리 정의된 tool들을 보려면 Tools documentation을 참고해라.</li>
        <li>Agent name: 지원되는 Agent Class를 참조하는 문자열이다. </li>
    </ol>
</ul>

In [6]:
# LLM
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

# 1. Agent를 컨트롤하기 위해 사용할 언어 모델 불러오기
llm = OpenAI(temperature=0)

# 2. Agent가 접근할 수 있게 할 tool. 참고로 'llm-math' tool은 LLM을 사용해서 LLM을 전달해줘야 한다.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# 3. Agent를 tool, 언어 모델, 사용할 agent의 type을 이용해서 초기화
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 4. 실행
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")


# Chat models
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# 1. Agent를 컨트롤하기 위해 사용할 언어 모델 불러오기
chat = ChatOpenAI(temperature=0)

# 2. Agent가 접근할 수 있게 할 tool. 참고로 'llm-math' tool은 LLM을 사용해서 LLM을 전달해줘야 한다.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# 3. Agent를 tool, 언어 모델, 사용할 agent의 type을 이용해서 초기화
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 4. 실행
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")



> Entering new AgentExecutor chain...
 I need to find the temperature first, then use the calculator to raise it to the .023 power.
Action: Search
Action Input: "High temperature in SF yesterday"
Observation: High: 68ºf @2:35 PM Low: 53.6ºf @2:45 AM Approx. Precipitation / Rain Total: in.
Thought: I now have the temperature, so I can use the calculator to raise it to the .023 power.
Action: Calculator
Action Input: 68^.023
Observation: Answer: 1.101914010425215
Thought: I now know the final answer.
Final Answer: 1.101914010425215

> Finished chain.


> Entering new AgentExecutor chain...
Thought: I need to use a search engine to find Olivia Wilde's boyfriend and a calculator to raise his age to the 0.23 power.
Action:
```
{
  "action": "Search",
  "action_input": "Olivia Wilde boyfriend"
}
```


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-EgY7FduP7DKCy2cOr4yEXVYB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..



Observation: In January 2021, Wilde began dating singer Harry Styles after meeting during the filming of Don't Worry Darling. Their relationship ended in November 2022.
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-EgY7FduP7DKCy2cOr4yEXVYB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-EgY7FduP7DKCy2cOr4yEXVYB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I need to use a calculator to raise Harry Styles' current age to the 0.23 power.
Action:
```
{
  "action": "Calculator",
  "action_input": "pow(28, 0.23)"
}
```


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-EgY7FduP7DKCy2cOr4yEXVYB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..



Observation: Answer: 2.1520202182226886
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-EgY7FduP7DKCy2cOr4yEXVYB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-EgY7FduP7DKCy2cOr4yEXVYB on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I now know the final answer.
Final Answer: Harry Styles' current age raised to the 0.23 power is approximately 2.152.

> Finished chain.


"Harry Styles' current age raised to the 0.23 power is approximately 2.152."

<H1>Memory</H1>

<ul>
    <li>Chain과 Agent는 stateless였다. 하지만 많은 애플리케이션들은 과거의 상호작용을 기억해야 한다.</li>
    <li>예를 들어서 챗봇은 과거의 메시지의 맥락을 토대로 새로운 메시지를 이해해야 한다.</li>
</ul>

<ul>
    <li>Memory module은 애플리케이션의 상태(state)를 유지할 방법을 준다.</li>
    <li>Memory interface의 기반은 간단하다: 가장 최근에 실행된 입력과 출력을 바탕으로 상태를 업데이트 할 수 있게 해주고, 저장된 상태를 이용해서 다음 입력을 수정(또는 맥락화)할 수 있게 해준다.</li>
    <li>다양한 내장 memory system이 있다. 가장 간단한 것은 buffer memory로, 현재의 input 앞에 마지막 몇 개의 입력/출력을 추가해준다.</li>
</ul>

In [7]:
# LLM
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.run("Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. My name is AI. What's your name?"

In [8]:
conversation.run("I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. My name is AI. What's your name?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"